In [1]:
#Average price of residential property
#(120 square meter apartments located in the most important cities) in Europe as of September 2020, per country

In [2]:
import pandas as pd
import numpy as np

In [3]:
#Primera fuente de CSV

df1 = pd.read_excel("average_residetial.xlsx","Data")


In [4]:
df1 = df1.fillna('-') #Para sustituir los NaNs

In [5]:
df1 = df1.rename(columns={'Unnamed: 0':'indice', 'Unnamed: 1': 'paises', 'Unnamed: 2': 'precio_m2'})

In [6]:
df1 = df1.drop([0,1,2,3], axis=0)

In [7]:
df1.head()

,indice,paises,precio_m2
4,-,Monaco,44522
5,-,UK,21179
6,-,Switzerland,13280
7,-,France,12796
8,-,Austria,9609


In [8]:
df1 = df1.drop('indice', axis=1)

In [54]:
df1.head()

,paises,precio_m2
4,Monaco,44522
5,UK,21179
6,Switzerland,13280
7,France,12796
8,Austria,9609


In [10]:
#Segunda fuente de CSV


In [11]:
df2 = pd.read_excel("rental.xlsx","Data")

In [12]:
df2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,NaN,Average rental cost of apartments in European ...,NaN,NaN
2,NaN,Average monthly rental cost of a furnished one...,NaN,NaN
3,NaN,NaN,Q4 2019,Q4 2020
4,NaN,Paris,1708,1803


In [13]:
df2 = df2.fillna('-')

In [14]:
df2 = df2.rename(columns={'Unnamed: 0':'indice', 'Unnamed: 1': 'ciudades', 'Unnamed: 2': 'alquiler_2019',
                          'Unnamed: 3': 'alquiler_2020'})

In [15]:
df2 = df2.drop([0,1,2,3], axis=0)

In [16]:
df2 = df2.drop('indice', axis=1)

In [17]:
df2
df2_lst = ["France","UK","Netherlands","Germany","Finland","Netherlands","Netherlands","Netherlands",
           "Spain","Italy","Portugal","Italy","Spain","Germany","Iceland","Germany","Germany","Belgium","Austria","Italy","Spain","Italy"]
df2["paises"]= df2_lst

In [18]:
df2.head()

,ciudades,alquiler_2019,alquiler_2020,paises
4,Paris,1708,1803,France
5,London,1747,1780,UK
6,Amsterdam,1683,1635,Netherlands
7,Munich,1525,1451,Germany
8,Helsinki,1398,1381,Finland


In [19]:
dfinal= pd.merge(df1, df2, left_on='paises', right_on='paises')

In [20]:
dfinal.head()

,paises,precio_m2,ciudades,alquiler_2019,alquiler_2020
0,UK,21179,London,1747,1780
1,France,12796,Paris,1708,1803
2,Austria,9609,Vienna,971,951
3,Finland,8376,Helsinki,1398,1381
4,Netherlands,6902,Amsterdam,1683,1635


In [21]:
backup=dfinal.copy()

In [22]:

dfinal = dfinal[["paises", "ciudades", "precio_m2", "alquiler_2019", "alquiler_2020"]]

In [23]:
dfinal

,paises,ciudades,precio_m2,alquiler_2019,alquiler_2020
0,UK,London,21179,1747,1780
1,France,Paris,12796,1708,1803
2,Austria,Vienna,9609,971,951
3,Finland,Helsinki,8376,1398,1381
4,Netherlands,Amsterdam,6902,1683,1635
5,Netherlands,Rotterdam,6902,1297,1306
6,Netherlands,Utrecht,6902,1289,1304
7,Netherlands,The Hague,6902,1212,1255
8,Italy,Milan,6589,1180,1141
9,Italy,Rome,6589,1114,1110


In [24]:
#Hacer columna de la variación y variación %

In [25]:
#Web Scrapping

In [26]:
from selenium import webdriver

In [27]:
webdriver.Chrome('driver/chromedriver_mac')

<selenium.webdriver.chrome.webdriver.WebDriver (session="fdf6fcb19f3246783847b8a6ab367d39")>

In [28]:
PATH ='driver/chromedriver_mac'

url ='https://www.reinisfischer.com/average-monthly-salary-european-union-2020'

In [29]:
import time

In [30]:
driver=webdriver.Chrome(PATH)


driver.get(url)

time.sleep(4)

In [31]:
ndf=driver.find_elements_by_tag_name('td')

In [32]:
lst_scr=[e.text for e in driver.find_elements_by_tag_name('tr')if len(e.text)> 3][1:]

In [33]:
df3 = pd.DataFrame([e.split() for e in lst_scr])

In [34]:
df3.head()

,0,1,2,3,4,5,6,7
0,Rank,Country,Gross,Gross,Change,EUR,Change,%
1,1,Denmark,"€5,179.00","€5,191.00",-€12.00,-0.23%,None,None
2,2,Luxembourg,"€5,143.00","€4,412.00",€731.00,16.57%,None,None
3,3,Germany,"€4,035.00","€3,703.00",€332.00,8.97%,None,None
4,4,Ireland,"€3,867.00","€3,133.00",€734.00,23.43%,None,None


In [57]:
col=[]
for i in df3.iloc[0]:
    col.append(i)


In [58]:
df3.columns = col

In [59]:
#df3.drop([0], axis=0)

In [60]:
#df1.rename(columns={'0':'indice', 'Unnamed: 1': 'paises', 'Unnamed: 2': 'precio_m2'})

In [61]:
df3.head()

,Rank,Country,Gross,Gross,Change,EUR,Change,%
0,Rank,Country,Gross,Gross,Change,EUR,Change,%
1,1,Denmark,"€5,179.00","€5,191.00",-€12.00,-0.23%,None,None
2,2,Luxembourg,"€5,143.00","€4,412.00",€731.00,16.57%,None,None
3,3,Germany,"€4,035.00","€3,703.00",€332.00,8.97%,None,None
4,4,Ireland,"€3,867.00","€3,133.00",€734.00,23.43%,None,None


In [62]:
#dframe= pd.merge(dfinal, df3, left_on='paises', right_on='Country')